# [Module 2.1] 세이지 메이커 로컬 모드 및 스크립트 모드로 훈련

본 워크샵의 모든 노트북은 **<font color="red">conda_tensorflow2_p36</font>** 를 사용합니다.

이 노트북은 아래와 같은 작업을 합니다.
- 1. 기본 환경 세팅 
- 2. 노트북에서 세이지 메이커 스크립트 모드 스타일로 코드 변경
- 3. 세이지 메이커 로컬 모드로 훈련
- 4. 세이지 메이커의 호스트 모드로 훈련
- 5. 모델 아티펙트 경로 저장


- 이 노트북은 아래와 같이 훈련 잡이 실행되는 "관리형 EC2 인스턴스" 의 저장 공간을 확인할 수 있는 "함수"를 훈련 스크립트에 추가했습니다.
    - 훈련시에 아래와 같은 저장 정보를 얻을 수 있습니다.
    - 자세한 내용은 이 훈련 코드를 참조 하세요. --> src/cifar10_tf2_sm_disk_size.py

![file-system-usage.png](images/file-system-usage.png)
---

# 1. 기본 환경 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-pytorch-cnn-cifar10"

role = sagemaker.get_execution_role()


In [3]:
import tensorflow as tf
print("tensorflow version: ", tf.__version__)

tensorflow version:  2.4.1


In [4]:
%store -r train_dir
%store -r validation_dir
%store -r eval_dir
%store -r data_dir

# 2. 노트북에서 세이지 메이커 스크립트 모드 스타일로 코드 변경

- Keras 버전의 스크래치 코드에서 세이지 메이커의 코드 변경을 참고 하세요.
    - `1.2.Train_Keras_Local_Script_Mode.ipynb` 참고


In [5]:
# !pygmentize src/cifar10_tf2_sm.py

# 3. 세이지 메이커 로컬 모드로 훈련

본격적으로 학습을 시작하기 전에 로컬 모드를 사용하여 디버깅을 먼저 수행합니다. 로컬 모드는 학습 인스턴스를 생성하는 과정이 없이 로컬 인스턴스로 컨테이너를 가져온 후 곧바로 학습을 수행하기 때문에 코드를 보다 신속히 검증할 수 있습니다.

Amazon SageMaker Python SDK의 로컬 모드는 TensorFlow 또는 MXNet estimator서 단일 인자값을 변경하여 CPU (단일 및 다중 인스턴스) 및 GPU (단일 인스턴스) SageMaker 학습 작업을 에뮬레이션(enumlate)할 수 있습니다. 

로컬 모드 학습을 위해서는 docker-compose 또는 nvidia-docker-compose (GPU 인스턴스인 경우)의 설치가 필요합니다. 아래 코드 셀을 통해 본 노트북 환경에 docker-compose 또는 nvidia-docker-compose를 설치하고 구성합니다. 
 
로컬 모드의 학습을 통해 여러분의 코드가 현재 사용 중인 하드웨어를 적절히 활용하고 있는지 확인하기 위한 GPU 점유와 같은 지표(metric)를 쉽게 모니터링할 수 있습니다.

### 로컬 모드로 훈련 실행
- 아래의 두 라인이 로컬모드로 훈련을 지시 합니다.
```python
    instance_type=instance_type, # local_gpu or local 지정
    session = sagemaker.LocalSession(), # 로컬 세션을 사용합니다.
```

#### 로컬의 GPU, CPU 여부로 instance_type 결정

In [6]:
import os
import subprocess


instance_type = "local_gpu" # GPU 사용을 가정 합니다. CPU 사용시에 'local' 로 정의 합니다.

print("Instance type = " + instance_type)

Instance type = local_gpu


학습 작업을 시작하기 위해 `estimator.fit() ` 호출 시, Amazon ECS에서 Amazon SageMaker TensorFlow 컨테이너를 로컬 노트북 인스턴스로 다운로드합니다.

`sagemaker.tensorflow` 클래스를 사용하여 SageMaker Python SDK의 Tensorflow Estimator 인스턴스를 생성합니다.
인자값으로 하이퍼파라메터와 다양한 설정들을 변경할 수 있습니다.


자세한 내용은 [documentation](https://sagemaker.readthedocs.io/en/stable/using_tf.html#training-with-tensorflow-estimator)을 확인하시기 바랍니다.

In [10]:
hyperparameters = {
                    'epochs' : 1,
                    'learning-rate' : 0.001,
                    'print-interval' : 100,
                    'train-batch-size': 256,    
                    'eval-batch-size': 512,        
                    'validation-batch-size': 512,
                  }

In [13]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_tf2_sm_disk_size.py',
                       source_dir='src',
                       role=role,
                       framework_version='2.4.1',
                       py_version='py37',
                       script_mode=True,
                       hyperparameters= hyperparameters,
                       train_instance_count=1, 
                       train_instance_type= instance_type)

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [14]:
%%time
estimator.fit({'train': f'file://{train_dir}',
               'validation': f'file://{validation_dir}',
               'eval': f'file://{eval_dir}'})

Creating 0rne89x5ps-algo-1-3ne7i ... 
Creating 0rne89x5ps-algo-1-3ne7i ... done
Attaching to 0rne89x5ps-algo-1-3ne7i
0rne89x5ps-algo-1-3ne7i | 2021-12-08 08:52:43.352032: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
0rne89x5ps-algo-1-3ne7i | 2021-12-08 08:52:43.352186: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
0rne89x5ps-algo-1-3ne7i | 2021-12-08 08:52:43.356455: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
0rne89x5ps-algo-1-3ne7i | 2021-12-08 08:52:43.393397: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
0rne89x5ps-algo-1-3ne7i | 2021-12-08 08:52:45,218 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
0rne89x5ps-algo

# 4. 세이지 메이커의 호스트 모드로 훈련

### 데이터 세트를 S3에 업로드


In [15]:
dataset_location = sagemaker_session.upload_data(path=data_dir, key_prefix='data/DEMO-cifar10')
display(dataset_location)

's3://sagemaker-us-east-1-057716757052/data/DEMO-cifar10'

In [17]:
hyperparameters = {
                    'epochs' : 20,
                    'learning-rate' : 0.001,    
                    'print-interval' : 100,
                    'train-batch-size': 256,    
                    'eval-batch-size': 512,        
                    'validation-batch-size': 512,
                  }

In [18]:
from sagemaker.tensorflow import TensorFlow

instance_type='ml.p3.8xlarge'

sm_estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_tf2_sm_disk_size.py',
                       source_dir='src',
                       role=role,
                       framework_version='2.4.1',
                       py_version='py37',
                       script_mode=True,
                       hyperparameters= hyperparameters,
                       train_instance_count=1, 
                       train_instance_type= instance_type)



train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


## SageMaker Host Mode 로 훈련
- `cifar10_estimator.fit(inputs, wait=False)`
    - 입력 데이터를 inputs로서 S3 의 경로를 제공합니다.
    - wait=False 로 지정해서 async 모드로 훈련을 실행합니다. 
        - 실행 경과는 아래의 cifar10_estimator.logs() 에서 확인 합니다.

In [19]:
%%time
sm_estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)}, wait=False)

CPU times: user 159 ms, sys: 0 ns, total: 159 ms
Wall time: 532 ms


In [20]:
sm_estimator.logs()

2021-12-08 08:55:33 Starting - Starting the training job...
2021-12-08 08:55:56 Starting - Launching requested ML instancesProfilerReport-1638953732: InProgress
.........
2021-12-08 08:57:28 Starting - Preparing the instances for training......
2021-12-08 08:58:18 Downloading - Downloading input data...
2021-12-08 08:58:56 Training - Downloading the training image.................2021-12-08 09:01:41.518525: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-12-08 09:01:41.522731: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-12-08 09:01:41.604479: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-12-08 09:01:41.696620: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler

# 5. 모델 아티펙트 저장
- S3 에 저장된 모델 아티펙트를 저장하여 추론시 사용합니다.

In [15]:
tf2_script_artifact_path = sm_estimator.model_data
print("script_tf_artifact_path: ", tf2_script_artifact_path)

%store tf2_script_artifact_path

script_tf_artifact_path:  s3://sagemaker-us-east-1-227612457811/cifar10-2021-10-11-11-22-24-452/output/model.tar.gz
Stored 'tf2_script_artifact_path' (str)
